In [1]:
import pandas as pd
import os

In [2]:
df_25k = pd.DataFrame()

for file_name in os.listdir("./raw dataset/25k-0914"):
    if file_name.endswith("json"):
        print("file path:", os.path.join("./raw dataset/25k-0914", file_name))
        tmp_df = pd.read_json(os.path.join("./raw dataset/25k-0914", file_name))
        df_25k = pd.concat([df_25k, tmp_df])

print(df_25k.shape)

file path: ./raw dataset/25k-0914\12k_test_context_0_6000.json
file path: ./raw dataset/25k-0914\12k_test_context_6000_last.json
file path: ./raw dataset/25k-0914\13k_test_context_0_7000.json
file path: ./raw dataset/25k-0914\13k_test_context_7000_last.json
(25395, 8)


In [3]:
import random

df_25k = df_25k.reset_index(drop=True)
df_shuffle = df_25k.copy()


def shuffle_answer(idx):
    try:
        tmp = ['A', 'B', 'C', 'D', 'E']
        random.shuffle(tmp)
        hm = {k: v for k, v in zip(['A', 'B', 'C', 'D', 'E'], tmp)}
        hm_b = {v: k for k, v in zip(['A', 'B', 'C', 'D', 'E'], tmp)}
        df_shuffle.loc[idx, ['A', 'B', 'C', 'D', 'E']] = [
            df_25k.loc[idx, hm[k]] for k in ['A', 'B', 'C', 'D', 'E']]
        df_shuffle.loc[idx, 'answer'] = hm_b[df_25k.loc[idx, 'answer']]
    except:
        print("Error!")


for i in range(len(df_shuffle)):
    shuffle_answer(i)

df_25k = df_shuffle.copy()
df_25k["answer"].value_counts()

answer
A    5182
D    5118
E    5079
B    5050
C    4966
Name: count, dtype: int64

In [4]:
df_25k.head()

,question,context,A,B,C,D,E,answer
0,What is the highest-charting song for Sparks o...,The song has become the highest- charting for ...,Kimono My House,Sugar Baby Love,None of the above,This Town Ain't Big Enough for Both of Us,The Rubettes,D
1,What is the main theme of The Wits?,"Common themes include sarcasm, tongue-in-cheek...",Religious conflicts,Love and romance,Political intrigue,Comedy and wit,Social class and hierarchy,D
2,What is the name of the debut album by The Ans...,Rise is the debut album by Northern Ireland ro...,Northern Irish,Downpatrick,County Down,Rise,Newcastle,D
3,How is Tamm crater related to Chaplygin crater?,It is located to the west-northwest of the muc...,Tamm is located to the east of Chaplygin.,Tamm is located to the north of Chaplygin.,Tamm is located inside Chaplygin.,Tamm is located to the west-northwest of Chapl...,Tamm is located to the south of Chaplygin.,D
4,What is the Ship of the Dead?,The novel was awarded the 2017 Stonewall Book ...,A threat to Asgard,A homeless orphan,A ship sailed by Magnus Chase and his crew,The third and final novel in the Magnus Chase ...,A young-adult fantasy novel based on Norse myt...,C


In [5]:
df_25k_with_nan = df_25k[df_25k.isna().any(axis=1)]
print(df_25k_with_nan.shape)

(0, 8)


In [6]:
df_25k.rename(columns={"question": "prompt"}, inplace=True)
df_25k.head()

,prompt,context,A,B,C,D,E,answer
0,What is the highest-charting song for Sparks o...,The song has become the highest- charting for ...,Kimono My House,Sugar Baby Love,None of the above,This Town Ain't Big Enough for Both of Us,The Rubettes,D
1,What is the main theme of The Wits?,"Common themes include sarcasm, tongue-in-cheek...",Religious conflicts,Love and romance,Political intrigue,Comedy and wit,Social class and hierarchy,D
2,What is the name of the debut album by The Ans...,Rise is the debut album by Northern Ireland ro...,Northern Irish,Downpatrick,County Down,Rise,Newcastle,D
3,How is Tamm crater related to Chaplygin crater?,It is located to the west-northwest of the muc...,Tamm is located to the east of Chaplygin.,Tamm is located to the north of Chaplygin.,Tamm is located inside Chaplygin.,Tamm is located to the west-northwest of Chapl...,Tamm is located to the south of Chaplygin.,D
4,What is the Ship of the Dead?,The novel was awarded the 2017 Stonewall Book ...,A threat to Asgard,A homeless orphan,A ship sailed by Magnus Chase and his crew,The third and final novel in the Magnus Chase ...,A young-adult fantasy novel based on Norse myt...,C


In [7]:
import regex as re

def clean_string(s):
    text = re.sub(r'#+', '', s)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\(\)', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def use_sep(s):
    text = re.sub(r'#####', '####', s)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\(\)', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
df_25k["context"] = df_25k["context"].apply(clean_string)
df_25k.to_json("./final dataset all/25k 0914 no #.json", orient="records")

In [9]:
# df_25k["context"] = df_25k["context"].apply(use_sep)
# df_25k.to_json("./final dataset all/25k 0914 with #.json", orient="records")